In [9]:
import cyvcf2
import sys

In [23]:
def extract_decision(record: cyvcf2.Variant) -> str:
    """Extract the decision from the FORMAT/BD fields.
    It can take the values TP, FP, FN, or .
    """
    bd = record.format("BD")
    # bd is a numpy array, convert to list
    bd = bd.tolist()
    truth_decision = bd[0]
    query_decision = bd[1]

    if truth_decision == ".":
        if query_decision == ".":
            print(record, file=sys.stderr)
            raise ValueError("Both truth and query decisions are missing")
        return query_decision
    else:
        return truth_decision

In [18]:
p = "/data/scratch/projects/punim2009/NanoVarBench/results/assess/mutref/illumina/ATCC_17802__202309/ATCC_17802__202309.summary.annotated.vcf.gz"

In [21]:
vcf = cyvcf2.VCF(p)

In [24]:
data = []
for record in vcf:
    decision = extract_decision(record)
    dist = record.INFO.get("DIST")
    if dist is None:
        print(record, file=sys.stderr)
        raise ValueError("No DIST field found in the record")
    density = record.INFO.get("DEN")
    if density is None:
        print(record, file=sys.stderr)
        raise ValueError("No DEN field found in the record")
    data.append((record.CHROM, record.POS, decision, dist, density))

[('chromosome_1', 255698, 'TP', 60780, 15),
 ('chromosome_1', 255698, 'TP', 60780, 15),
 ('chromosome_1', 255699, 'TP', 60781, 16)]